## Play Ground

In [8]:
# Example in Chapter definition 4.10
import math
def calculate_uncertainty(probabilities, k):
    uncertainty = k * sum(p * math.log2(p) for p in probabilities)
    return uncertainty

# Example usage
probabilities = [0.7, 0.27, 0.03]
k = 1000
uncertainty = calculate_uncertainty(probabilities, k)
print("Uncertainty:", uncertainty)

# surprise: individual uncertainty for event 3
surprise = math.log2(probabilities[2])
print("Surprise:", surprise)

Uncertainty: -1021.989577307477
Surprise: -5.058893689053568


In [7]:
# Law 5.1

probabilities = [0.5, 0.3, 0.2]
k = 10000
uncertainty = calculate_uncertainty(probabilities, k)
print("Uncertainty:", uncertainty)

Uncertainty: -14854.752972273343


In [6]:
def counting_theorem(n, d):
    if n == 0 or d == 0:
        return 0
    elif n == 1 or d == 1:
        return 2
    else:
        return counting_theorem(n-1, d) + counting_theorem(n-1, d-1)
    
print(counting_theorem(4, 3))


14


### 6.1 (a) (b)

In [35]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def generate_random_points_and_labels(n_points, n_features, c_classes):
    # Generate random points with n_features
    np.random.seed(2)
    X = np.random.rand(n_points, n_features)
    # Generate random labels with c equi-distributed classes
    labels = np.random.choice(range(c_classes), n_points)
    return X, labels

def get_number_of_thresholds(data, labels):
    table = [(np.sum(row), label) for row, label in zip(data, labels)]
    sorted_table = sorted(table, key=lambda x: x[0])
    thresholds = 0
    previous_label = None
    for data_sum, label in sorted_table:
        if label != previous_label:
            previous_label = label
            thresholds += 1
    return thresholds

def train_nearest_neighbors(X_train, y_train):
    # Train a nearest neighbors classifier
    clf = KNeighborsClassifier(n_neighbors=1)  # Using 1-NN for simplicity
    clf.fit(X_train, y_train)
    return clf

def get_number_of_instances_memorized(X_test, y_test, clf):
    # Predict the labels for the test set
    y_pred = clf.predict(X_test)
    # For 1-NN, we'll consider the number of correctly classified instances as memorized instances
    memorized_instances = np.sum(y_pred == y_test)
    return memorized_instances

def main(n_points=10000, n_features=2, c_classes=3):
    # Step 1: Generate random points and labels
    X, labels = generate_random_points_and_labels(n_points, n_features, c_classes)

    # Step 2: Train the nearest neighbors classifier
    clf = train_nearest_neighbors(X, labels)

    # Step 3: Information Capacity
    thresholds = get_number_of_thresholds(X, labels)
    memorized_instances = get_number_of_instances_memorized(X, labels, clf)
    print(memorized_instances)
    info_capacity = memorized_instances / thresholds
    print(f"Information Capacity: {info_capacity:.3f} bits per parameter")

if __name__ == "__main__":
    main(n_points=128, n_features=10, c_classes=5)

# c/c-1 4/3

128
Information Capacity: 1.174 bits per parameter


In [21]:
## generate data X (n_points, n_features) 000 001 010 011 100 101 110 111
def generate_data(n_points, n_features):
    data = np.zeros((n_points, n_features))
    for i in range(n_points):
        for j in range(n_features):
            data[i, j] = i >> j & 1
    return data
generate_data(8, 3)

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.]])

In [87]:
import numpy as np
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

def generate_random_points(n_points, n_features):
    # X = np.random.rand(n_points, n_features)
    X = np.random.randint(0, 2, (n_points, n_features))
    return X

def generate_data(n_points, n_features):
    data = np.zeros((n_points, n_features))
    for i in range(n_points):
        for j in range(n_features):
            data[i, j] = i >> j & 1
    return data

def generate_random_labels(n_points, c_classes):
    labels = np.random.choice(range(c_classes), n_points)
    return labels

def generate_unique_random_labels(n_points, c_classes, n_labelings):
    unique_labelings = set()
    while len(unique_labelings) < n_labelings:
        # Generate a random labeling
        labels = tuple(np.random.choice(range(c_classes), n_points))
        unique_labelings.add(labels)

    # Convert each tuple back to a numpy array
    unique_labelings = [np.array(labels) for labels in unique_labelings]
    return unique_labelings

def condensed_nearest_neighbor(X, y):
    # Initialize S with one example from each class (optional)
    # unique_labels = np.unique(y)
    # S_indices = [np.where(y == label)[0][0] for label in unique_labels]
    S_indices = [0]
    changed = True
    while changed:
        changed = False
        for i in range(len(X)):
            if i in S_indices:
                continue  # Skip if already in S
            S = X[S_indices]
            S_labels = y[S_indices]
            knn = KNeighborsClassifier(n_neighbors=1)
            knn.fit(S, S_labels)
            pred = knn.predict([X[i]])[0]
            if pred != y[i]:
                S_indices.append(i)
                changed = True
    return len(S_indices)

def main(d, num_function, c_classes):
    n = 2**d
    avg_mem_size = 0
    X = generate_data(n, d)
    # Y = generate_unique_random_labels(n, c_classes, num_function)
    for i in range(num_function):
        labels = generate_random_labels(n, c_classes)
        req_points = condensed_nearest_neighbor(X, labels)
        avg_mem_size += req_points
    avg_mem_size /= num_function
    print(f"d={d}: n_full={2**d}, \
          Avg. req. points for memorization n_avg={avg_mem_size:.2f}, \
          n_full/n_avg={(2**d)/avg_mem_size}")

if __name__ == "__main__":
    main(d=2, num_function=16, c_classes=2)
    main(d=4, num_function=2**4, c_classes=2)
    main(d=6, num_function=2**6, c_classes=2)
    main(d=8, num_function=2**8, c_classes=2)



d=2: n_full=4,           Avg. req. points for memorization n_avg=2.69,           n_full/n_avg=1.4883720930232558
d=4: n_full=16,           Avg. req. points for memorization n_avg=8.94,           n_full/n_avg=1.7902097902097902
d=6: n_full=64,           Avg. req. points for memorization n_avg=33.47,           n_full/n_avg=1.912231559290383


In [61]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def condensed_nearest_neighbor(X, y):
    # Initialize S with one example from each class (optional)
    unique_labels = np.unique(y)
    S_indices = [np.where(y == label)[0][0] for label in unique_labels]
    
    changed = True
    while changed:
        changed = False
        for i in range(len(X)):
            if i in S_indices:
                continue  # Skip if already in S
            S = X[S_indices]
            S_labels = y[S_indices]
            knn = KNeighborsClassifier(n_neighbors=1)
            knn.fit(S, S_labels)
            pred = knn.predict([X[i]])[0]
            if pred != y[i]:
                S_indices.append(i)
                changed = True
                
    return X[S_indices], y[S_indices]

# Example use
# Generate random data
d = 10
n = 2 ** d
X = np.random.rand(n, d)
y = np.random.choice([0, 1], n)  # Ensure y has the same length as X

X_condensed, y_condensed = condensed_nearest_neighbor(X, y)
print(f"Original dataset size: {len(X)}, Condensed dataset size: {len(X_condensed)}")

# Test the condensed dataset
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_condensed, y_condensed)
print(f"Accuracy on original dataset: {knn.score(X, y):.2f}")

Original dataset size: 1024, Condensed dataset size: 691
Accuracy on original dataset: 1.00


### 6.2 (c)

In [9]:
import numpy as np
import pandas as pd

def generate_random_binary_dataset(n_samples=100, n_features=5):
    X = np.random.randint(2, size=(n_samples, n_features))
    y = np.random.randint(2, size=n_samples)
    return X, y

X, y = generate_random_binary_dataset()
df = pd.DataFrame(X, columns=[f'Feature_{i}' for i in range(X.shape[1])])
df['Target'] = y


In [10]:
from itertools import combinations

def prune_redundant_rules(rules):
    pruned_rules = []
    for rule in rules:
        if not any(all(item in other_rule for item in rule) for other_rule in rules if rule != other_rule):
            pruned_rules.append(rule)
    return pruned_rules

def combine_similar_rules(rules):
    combined_rules = []
    for rule_a, rule_b in combinations(rules, 2):
        diff = [item for item in rule_a if item not in rule_b] + [item for item in rule_b if item not in rule_a]
        if len(diff) == 2 and diff[0][:-2] == diff[1][:-2]:  # Only one condition differs
            new_rule = tuple(set(rule_a).union(set(rule_b)) - set(diff))
            if new_rule not in combined_rules:
                combined_rules.append(new_rule)
    return combined_rules + [rule for rule in rules if rule not in combined_rules]

def rule_ordering_and_early_stopping(rules, dataset):
    # Sort rules by coverage (number of samples it applies to)
    rules_coverage = {rule: sum(all(row[feature] == value for feature, value in rule[:-1]) for _, row in dataset.iterrows()) for rule in rules}
    sorted_rules = sorted(rules, key=lambda rule: -rules_coverage[rule])
    # Early stopping is implemented during rule evaluation, not here
    return sorted_rules

def evaluate_rules(rules, dataset):
    correct_predictions = 0
    for _, row in dataset.iterrows():
        for rule in rules:
            if all(row[feature] == value for feature, value in rule[:-1]):
                correct_predictions += rule[-1] == row['Target']
                break  # Early stopping
    accuracy = correct_predictions / len(dataset)
    return accuracy

# Generate all possible if-then clauses (rules) from the dataset
all_rules = [(tuple(zip(row.index[:-1], row.values[:-1])), row['Target']) for _, row in df.iterrows()]

# Apply strategies
pruned_rules = prune_redundant_rules(all_rules)
combined_rules = combine_similar_rules(pruned_rules)
ordered_rules = rule_ordering_and_early_stopping(combined_rules, df)

# Evaluate the rules
accuracy = evaluate_rules(ordered_rules, df)
print(f"Number of rules after minimization: {len(ordered_rules)}")
print(f"Accuracy: {accuracy}")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### 6.3 (a)

In [14]:
import zlib
import numpy as np

# Generate a long random string
random_string = ''.join(np.random.choice(['A', 'C', 'G', 'T'], size=100))

# Convert the string to bytes
random_string_bytes = random_string.encode('utf-8')

# Compress the string using zlib
compressed_string = zlib.compress(random_string_bytes)

# Calculate the compression ratio
compression_ratio = len(compressed_string) / len(random_string_bytes)

compression_ratio


0.59

The expected compression ratio for a random string, especially one composed of a small set of characters (like 'A', 'C', 'G', 'T' in the case of our DNA sequence example), can vary significantly depending on several factors, including the length of the string, the distribution of characters, and the compression algorithm used. However, random data or sequences with high entropy (where each character is equally likely and independent of the others) are generally hard to compress effectively because the essence of lossless compression is to find and exploit patterns, redundancy, or predictability in the data.

For the example given, where the string is generated from an equal distribution of four characters, we might expect a lower compression ratio than highly redundant or patterned data. This is because, even though there are repeating characters, the randomness and uniform distribution reduce the predictability and hence the compressibility of the sequence. 

Compression algorithms like zlib implement a combination of strategies (such as dictionary-based compression in DEFLATE, which zlib uses) to reduce data size. These strategies work best when the data has noticeable patterns or redundancy. In the case of a truly random sequence, the compression algorithm might struggle to find useful patterns to exploit, leading to less significant compression. 

The observed compression ratio of approximately 0.3195 suggests that zlib was able to find some patterns or redundancies to exploit, possibly due to the limited character set, even though the sequence was intended to be random. This ratio might not be as good as for data with more obvious patterns (like repeated sequences or large blocks of identical characters) but is still notable given the randomness of the input.

It's important to note that for truly random data, the theoretical best compression you could expect (without losing information) is very minimal, due to the Shannon entropy of the data being high. In practical terms, this means that the more random and less predictable the data, the closer the compression ratio will approach 1 (no compression), after accounting for the overhead of the compression algorithm itself.

In [68]:
import numpy as np

def memorize(data, labels):
    """
    Function to calculate the minimum encoding cost (mec) for given data and labels.
    
    Parameters:
    data (np.array): An array of n d-dimensional vectors.
    labels (np.array): A column of 0s or 1s with length n.
    
    Returns:
    float: The minimum encoding cost (mec).
    """
    # Calculate the sum for each row and pair it with its label
    table = [(np.sum(row), label) for row, label in zip(data, labels)]
    
    # Sort the table based on the sum (column 0)
    sorted_table = sorted(table, key=lambda x: x[0])
    
    # Initialize variables
    thresholds = 0
    class_ = 0
    
    # Determine the number of threshold changes
    for row in sorted_table:
        if row[1] != class_:
            class_ = row[1]
            thresholds += 1
    # Calculate the minimum number of thresholds
    min_threshs = np.log2(thresholds + 1)
    
    # Calculate minimum encoding cost
    mec = (min_threshs * (data.shape[1] + 1)) + (min_threshs + 1)
    
    return mec

# Example usage
n = 100
d = 5
np.random.seed(42)
data = np.random.rand(n, d)  # n d-dimensional vectors
labels = np.random.randint(2, size=n)  # n binary labels
mec = memorize(data, labels)
mec

40.30296890880646

In [67]:
# Modification for multi-class labels

def memorize_multiclass(data, labels):
    """
    Function to calculate the minimum encoding cost (mec) for given data and labels for multi-class classification.
    
    Parameters:
    data (np.array): An array of n d-dimensional vectors.
    labels (np.array): A column of labels with length n, for multi-class classification.
    
    Returns:
    float: The minimum encoding cost (mec).
    """
    # Calculate the sum for each row and pair it with its label
    table = [(np.sum(row), label) for row, label in zip(data, labels)]
    
    # Sort the table based on the sum (column 0)
    sorted_table = sorted(table, key=lambda x: x[0])
    
    # Initialize variables
    thresholds = 0
    previous_label = None
    
    # Determine the number of threshold changes
    for data_sum, label in sorted_table:
        if label != previous_label:
            previous_label = label
            thresholds += 1
    # Calculate the minimum number of thresholds
    min_threshs = np.log2(thresholds + 1)
    
    # Calculate minimum encoding cost
    d = data.shape[1]
    mec = (min_threshs * (d + 1)) + (min_threshs + 1)
    
    return mec

# Example usage with multi-class labels
n = 100
d = 5
np.random.seed(42)
data = np.random.rand(n, d)  # n d-dimensional vectors
labels = np.random.randint(2, size=n)  # n labels for multi-class classification

mec_multiclass = memorize_multiclass(data, labels)
mec_multiclass


40.30296890880646

### 9.1

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# Define a flexible CNN model
class CustomCNN(nn.Module):
    def __init__(self, conv_layers, linear_layers, dropout_rate=0.5):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()

        # Add convolutional layers
        for in_channels, out_channels, kernel_size, stride, padding in conv_layers:
            self.layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))
            self.layers.append(nn.BatchNorm2d(out_channels))
            self.layers.append(nn.ReLU(inplace=True))
            self.layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

        self.layers.append(nn.Flatten())

        # Add linear layers
        for in_features, out_features in linear_layers[:-1]:
            self.layers.append(nn.Linear(in_features, out_features))
            self.layers.append(nn.ReLU(inplace=True))
            self.layers.append(nn.Dropout(dropout_rate))

        # Add the final linear layer without dropout
        in_features, out_features = linear_layers[-1]
        self.layers.append(nn.Linear(in_features, out_features))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Hyperparameters and model configuration - easily modifiable
conv_layers = [
    (1, 32, 3, 1, 1), # in_channels, out_channels, kernel_size, stride, padding
    (32, 64, 3, 1, 1),
]

linear_layers = [
    (7*7*64, 128), # in_features, out_features
    (128, 10),
]

# Data preparation
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CustomCNN(conv_layers, linear_layers, dropout_rate=0.5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

# Run training and evaluation
train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader)


100%|██████████| 9912422/9912422 [00:00<00:00, 43409094.34it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 104698093.19it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30168781.18it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14195625.01it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Loss: 0.23937382253963174
Epoch 2, Loss: 0.11206717976728386
Epoch 3, Loss: 0.08702044609089149
Epoch 4, Loss: 0.07455138549499556
Epoch 5, Loss: 0.0684255073962759
Epoch 6, Loss: 0.05826709370372315
Epoch 7, Loss: 0.05079960703640295
Epoch 8, Loss: 0.046488662213417214
Epoch 9, Loss: 0.043231522153807565
Epoch 10, Loss: 0.039125274848761354
Accuracy: 99.24%


In [20]:
import torch

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = count_parameters(model)
print(f"Total trainable parameters in the model: {total_params}")

Total trainable parameters in the model: 421834


In [21]:
def calculate_parameters(conv_layers, linear_layers):
    total_params = 0
    # Calculate parameters for convolutional layers
    for in_channels, out_channels, kernel_size, stride, padding in conv_layers:
        conv_params = out_channels * (in_channels * kernel_size * kernel_size + 1)  # +1 for bias
        bn_params = 2 * out_channels  # BatchNorm has running_mean and running_var
        total_params += conv_params + bn_params
    # Calculate parameters for linear layers
    for in_features, out_features in linear_layers:
        linear_params = out_features * (in_features + 1)  # +1 for bias
        total_params += linear_params
    return total_params

calculate_parameters(conv_layers, linear_layers)

421834

In [77]:
def count_neurons(model):
    neuron_count = 0
    for layer in model.layers:
        if isinstance(layer, nn.Linear):
            # For linear layers, the number of out_features is the number of neurons
            neuron_count += layer.out_features
        elif isinstance(layer, nn.Conv2d):
            # For Conv2d layers, use the number of out_channels as an approximation of neurons
            neuron_count += layer.out_channels
        elif isinstance(layer, nn.MaxPool2d):
            # Optional: Include pooling layers if considering them as having neurons
            # Note: This would be an approximation and depends on input size
            pass
    return neuron_count

# Example usage with the CustomCNN model
# model = CustomCNN(conv_layers, linear_layers)
print(f"The number of neurons in the model: {count_neurons(model)}")


The number of neurons in the model: 234


In [23]:
import numpy as np

def dataset_to_ndarray(dataset):
    data = []
    labels = []
    
    for image, label in dataset:
        # Convert the image to a NumPy array and flatten it
        np_image = image.numpy().flatten()
        data.append(np_image)
        
        # Append the label
        labels.append(label)
    
    # Convert lists to NumPy arrays
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

# Convert train_dataset to data and labels NumPy arrays
data, labels = dataset_to_ndarray(train_dataset)

print(f"data shape: {data.shape}, labels shape: {labels.shape}")


data shape: (60000, 784), labels shape: (60000,)


In [25]:
mec = memorize(data, labels)
print(f"Memory-Equivalent Capacity (MEC): {mec}")

Memory-Equivalent Capacity (MEC): 12362.058469281907
